---
title: Évaluation des classifieurs
---

### Introduction

Ce document présente les résultats des différents classifieurs binaire sur les 28 jeux de données fournis. 
Les résultats présenter dans ce document sont aggrégés sur les 28 jeux de données. Les résultats détaillés pour chaque jeu de données sont disponibles dans les fichiers respectif. Les résultats individuels permettent notamment d'évaluer 

In [ ]:
from pathlib import Path
import pandas as pd
from joblib import load

#### Évaluation des classifieurs binaires

In [2]:
import sys
sys.path.append("./../src/")

from get_dataset import dataset_loaders
dataset = list(dataset_loaders.keys())[0]

La [](#table_report_LR1) montre les résultats de la classification par le modèle de régression logistique. On observe que :

- $83,04 \%$ des *spams* sont correctement identifiés
- $99,59 \%$ des *hams* sont correctement identifiés
- $96,88 \%$ des observations classifiées en tant que *spam* sont des *spams*
- $97,43 \%$ des observations classifiées en tant que *ham* sont des *hams*
- Le score F1 moyen pondéré est de $97,98 \%$ 